## AutoML-EvalML サンプルコード

In [1]:
%load_ext lab_black

In [2]:
# ライブラリーのインポート
import os

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# ボストンの住宅価格データ
from sklearn.datasets import load_boston

# 前処理
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# evalml
import evalml
from evalml import AutoMLSearch

# 評価指標
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [3]:
print(evalml.__version__)

0.31.0


In [4]:
# データセットの読込み
boston = load_boston()

# 説明変数の格納
df = pd.DataFrame(boston.data, columns=boston.feature_names)
# 目的変数の追加
df["MEDV"] = boston.target

# データの中身を確認
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [5]:
# ランダムシード値
RANDOM_STATE = 10

# 学習データと評価データの割合
TEST_SIZE = 0.2

# 学習データと評価データを作成
x_train, x_test, y_train, y_test = train_test_split(
    df.iloc[:, 0 : df.shape[1] - 1],
    df.iloc[:, df.shape[1] - 1],
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
)

In [6]:
automl = AutoMLSearch(
    X_train=x_train,
    y_train=y_train,
    problem_type="regression",
    objective="mae",
    additional_objectives=["R2", "mse"],
    max_time=120,
    random_seed=RANDOM_STATE,
    optimize_thresholds=True,
)
automl.search()

Generating pipelines to search over...
8 pipelines ready for search.

*****************************
* Beginning pipeline search *
*****************************

Optimizing for MAE. 
Lower score is better.

Using SequentialEngine to train and score pipelines.
Will stop searching for new pipelines after 120 seconds.

Allowed model families: extra_trees, lightgbm, xgboost, decision_tree, random_forest, linear_model, catboost



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Evaluating Baseline Pipeline: Mean Baseline Regression Pipeline


/tmp/Python/python-3.9.6/lib/python3.9/site-packages/evalml/pipelines/component_graph.py:135: ParameterNotUsedWarning:

Parameters for components {'Baseline Classifier'} will not be used to instantiate the pipeline since they don't appear in the pipeline



Mean Baseline Regression Pipeline:
	Starting cross validation
	Finished cross validation - mean MAE: 6.317

*****************************
* Evaluating Batch Number 1 *
*****************************

Linear Regressor w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean MAE: 3.251
Decision Tree Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean MAE: 2.843
Random Forest Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean MAE: 2.307
LightGBM Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean MAE: 2.675
Elastic Net Regressor w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean MAE: 3.250
XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean MAE: 2.133
Extra Trees Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean MAE: 2.241
CatBoost Regressor w/ Imputer:


In [7]:
automl.rankings

,id,pipeline_name,search_order,mean_cv_score,standard_deviation_cv_score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,17,Extra Trees Regressor w/ Imputer,17,2.110322,0.192494,1.983612,420.669337,False,{'Imputer': {'categorical_impute_strategy': 'm...
1,6,XGBoost Regressor w/ Imputer,6,2.132675,0.176276,1.985958,418.434048,False,{'Imputer': {'categorical_impute_strategy': 'm...
4,23,Random Forest Regressor w/ Imputer,23,2.201922,0.236735,2.063691,411.509372,False,{'Imputer': {'categorical_impute_strategy': 'm...
16,4,LightGBM Regressor w/ Imputer,4,2.675462,0.349858,2.428031,364.155364,False,{'Imputer': {'categorical_impute_strategy': 'm...
18,2,Decision Tree Regressor w/ Imputer,2,2.842775,0.105599,2.781685,347.424066,False,{'Imputer': {'categorical_impute_strategy': 'm...
20,5,Elastic Net Regressor w/ Imputer + Standard Sc...,5,3.250368,0.155666,3.171171,306.664790,False,{'Imputer': {'categorical_impute_strategy': 'm...
21,1,Linear Regressor w/ Imputer + Standard Scaler,1,3.250674,0.155452,3.171864,306.634228,False,{'Imputer': {'categorical_impute_strategy': 'm...
22,8,CatBoost Regressor w/ Imputer,8,5.327592,0.635506,4.595449,98.942423,False,{'Imputer': {'categorical_impute_strategy': 'm...
23,0,Mean Baseline Regression Pipeline,0,6.317016,0.776570,5.432474,0.000000,False,{'Baseline Regressor': {'strategy': 'mean'}}


In [8]:
automl.describe_pipeline(automl.rankings.iloc[0]["id"])


************************************
* Extra Trees Regressor w/ Imputer *
************************************

Problem Type: regression
Model Family: Extra Trees

Pipeline Steps
1. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : most_frequent
	 * categorical_fill_value : None
	 * numeric_fill_value : None
2. Extra Trees Regressor
	 * n_estimators : 584
	 * max_features : auto
	 * max_depth : 8
	 * min_samples_split : 2
	 * min_weight_fraction_leaf : 0.0
	 * n_jobs : -1

Training
Training for regression problems.
Total training time (including CV): 8.8 seconds

Cross Validation
----------------
              MAE    R2    MSE # Training # Validation
0           1.984 0.858  6.968        269          135
1           2.016 0.910  7.948        269          135
2           2.332 0.832 15.494        270          134
mean        2.110 0.867 10.137          -            -
std         0.192 0.040  4.666          -            -
coef of var 0.091 0.046  0.46

In [9]:
best_model = automl.best_pipeline
best_model.save("model.pkl")

In [10]:
# 推論
y_pred = best_model.predict(x_test)

In [11]:
# 評価
def calculate_scores(true, pred):
    """全ての評価指標を計算する

    Parameters
    ----------
    true (np.array)       : 実測値
    pred (np.array)       : 予測値

    Returns
    -------
    scores (pd.DataFrame) : 各評価指標を纏めた結果

    """
    scores = {}
    scores = pd.DataFrame(
        {
            "R2": r2_score(true, pred),
            "MAE": mean_absolute_error(true, pred),
            "MSE": mean_squared_error(true, pred),
            "RMSE": np.sqrt(mean_squared_error(true, pred)),
        },
        index=["scores"],
    )
    return scores

In [12]:
scores = calculate_scores(y_test, y_pred)
print(scores)

              R2       MAE        MSE      RMSE
scores  0.875305  2.599936  13.040664  3.611186
